#### 1. Import pandas library

In [25]:
import pandas as pd

#### 2. Import users table:

In [26]:
user = pd.read_csv('users_table.csv')

#### 3. Rename Id column to userId

In [59]:
user.rename(columns={'Id':'userId'},inplace = True)

#### 4. Import posts table:

In [78]:
posts = pd.read_csv('posts_table.csv')

#### 5. Rename Id column to postId and OwnerUserId to userId

In [80]:
posts.rename(columns={'Id':'postId','OwnerUserId':'userId'},inplace = True)
posts.head(5)

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaN,NaN,NaN,NaN,NaN
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaN,NaN,NaN
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,NaN,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaN,NaN,NaN


#### 6. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userId,ViewCount,CommentCount

In [83]:
users= user[['userId','Reputation','Views','UpVotes','DownVotes']]

In [84]:
posts= posts[['postId', 'Score','userId','ViewCount','CommentCount']]

#### 7. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [87]:
#merged both tabells on userId
merge_user_post=users.merge(posts,on='userId')

#### 8. How many missing values do you have in your merged dataframe? On which columns?

In [100]:
# count the number of null values per column
merge_user_post.isnull().sum(axis = 0)

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       23572
CommentCount        0
dtype: int64

#### 9. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [108]:
#fill Nulls with 0
merge_user_post.fillna(0,inplace=True)

#### 10. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [111]:
#See all data types per column
merge_user_post.info()

# Change the type of ViewCount to int, so that all are aligned 
merge_user_post= merge_user_post.astype({"ViewCount": int})

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38962 entries, 0 to 38961
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   userId        38962 non-null  int64  
 1   Reputation    38962 non-null  int64  
 2   Views         38962 non-null  int64  
 3   UpVotes       38962 non-null  int64  
 4   DownVotes     38962 non-null  int64  
 5   postId        38962 non-null  int64  
 6   Score         38962 non-null  int64  
 7   ViewCount     38962 non-null  float64
 8   CommentCount  38962 non-null  int64  
dtypes: float64(1), int64(8)
memory usage: 3.0 MB


In [112]:
merge_user_post.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 38962 entries, 0 to 38961
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   userId        38962 non-null  int64
 1   Reputation    38962 non-null  int64
 2   Views         38962 non-null  int64
 3   UpVotes       38962 non-null  int64
 4   DownVotes     38962 non-null  int64
 5   postId        38962 non-null  int64
 6   Score         38962 non-null  int64
 7   ViewCount     38962 non-null  int32
 8   CommentCount  38962 non-null  int64
dtypes: int32(1), int64(8)
memory usage: 2.8 MB
